In [28]:
import pandas as pd
import seaborn as sns
import math
from matplotlib import pyplot as plt
import matplotlib
import re
import time
from statistics import geometric_mean
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/tmp/ipykernel_21378/1427944705.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [29]:
f1 = 'F(a & F(b & F(c & F(d & XF(a)))))'
f2 = 'F(a & F(b & F(c & F(d & F(e & F(f & F(g & F(h & F(i & XF(a))))))))))'
f3 = '((~b)U(a & XF (b))) & ((~c)U(b & XF (c)))'
f4 = 'F(a & XF (b & XF(c|(d & XF e))))'
f5 = 'G(~d) & ((~(b|c)) U a) & ((~c) U b) & F(a & XF(b & XF(c)))'

In [30]:
winltl = dict()
winltla = dict()
ties = dict()
solgap = dict()

for h in ['h1', 'hm', 'hcp']:
    if h not in winltl:
        winltl[h] = dict()
    if h not in winltla:
        winltla[h] = dict()
    if h not in ties:
        ties[h] = dict()
    if h not in solgap:
        solgap[h] = dict()
    for domain in ['8', 'starcraft', 'maze32']:
        winltl[h][domain] = 0
        winltla[h][domain] = 0
        ties[h][domain] = 0
        solgap[h][domain] = []
        


for f in [f1,f2,f3,f4,f5]:
    for rep in ["rep3", 'rep25']:
        for domain in ['8', 'starcraft', 'maze32']:
            resultsh1 = []
            resultshm = []
            resultshcp = []
            for alg in ['LTL-LRTA*','LTL-LRTA*A']:
                name = f'G{domain} - h1 - {rep} - f{f} - {alg}.csv'
                try:
                    with open(name, 'r') as file:
                        lines = file.readlines()
                    resultsh1.append(pd.read_csv(name))
                except FileNotFoundError:
                    print("ERROR:", name)
                    break
                    
                name = f'G{domain} - hm - {rep} - f{f} - {alg}.csv'
                try:
                    with open(name, 'r') as file:
                        lines = file.readlines()
                    resultshm.append(pd.read_csv(name))
                except FileNotFoundError:
                    print("ERROR:", name)
                    break

                name = f'G{domain} - hcp - {rep} - f{f} - {alg}.csv'
                try:
                    with open(name, 'r') as file:
                        lines = file.readlines()
                    resultshcp.append(pd.read_csv(name))
                except FileNotFoundError:
                    print("ERROR:", name)
                    break
            
            datah1 = pd.concat(resultsh1).reset_index()
            datahm = pd.concat(resultshm).reset_index()
            datahcp = pd.concat(resultshcp).reset_index()
            datah1.drop('index', axis=1, inplace=True)
            datahm.drop('index', axis=1, inplace=True)
            datahcp.drop('index', axis=1, inplace=True)
            datah1.columns = ['Algorithm', 'Lookahead', 'Room', 'Map', 'Grid', 'Steps', 'Episodios', 'Expansiones', 'Microseconds']
            datahm.columns = ['Algorithm', 'Lookahead', 'Room', 'Map', 'Grid', 'Steps', 'Episodios', 'Expansiones', 'Microseconds']
            datahcp.columns = ['Algorithm', 'Lookahead', 'Room', 'Map', 'Grid', 'Steps', 'Episodios', 'Expansiones', 'Microseconds']
            datah1['Miliseconds'] = datah1['Microseconds']*1000
            datahm['Miliseconds'] = datahm['Microseconds']*1000
            datahcp['Miliseconds'] = datahcp['Microseconds']*1000
            datah1.drop('Microseconds', axis=1, inplace=True)
            datahm.drop('Microseconds', axis=1, inplace=True)
            datahcp.drop('Microseconds', axis=1, inplace=True)
            datah1['Heuristic'] = 'h1'
            datahm['Heuristic'] = 'hm'
            datahcp['Heuristic'] = 'hcp'
            data = []

            d = pd.concat([datahcp, datahm, datah1])
            d = d.reset_index().drop(columns=['index'])

            for h in ['h1', 'hm', 'hcp']:
                resltl = []
                resltla = []
                for k in sorted(list(set(d['Lookahead']))):
                    if k < 32:
                        continue

                    var1 = d[(d['Lookahead'] == k) & (d['Algorithm'] == "LTL-LRTA*") & (d['Heuristic'] == h)]['Steps']
                    var2 = d[(d['Lookahead'] == k) & (d['Algorithm'] == "LTL-LRTA*A") & (d['Heuristic'] == h)]['Steps']
                    for ltl, ltla in zip(var1, var2):
                        if ltl < ltla:
                            winltl[h][domain] += 1
                        elif ltla < ltl:
                            winltla[h][domain] += 1
                        else:
                            ties[h][domain] += 1
                        solgap[h][domain].append(ltl/ltla)

In [31]:
for h in ['h1', 'hm', 'hcp']:
    for domain in ['8', 'starcraft', 'maze32']:
        print(h, domain, winltl[h][domain], winltla[h][domain], ties[h][domain], geometric_mean(solgap[h][domain]))

h1 8 1020 1980 0 1.2946051890046408
h1 starcraft 1213 1787 0 1.157687684011814
h1 maze32 1272 1728 0 1.1368360552891088
hm 8 841 1524 635 1.0204463089014857
hm starcraft 461 531 2008 1.0021591991500207
hm maze32 588 696 1716 1.004561727424245
hcp 8 20 2980 0 31.912438282000537
hcp starcraft 27 2973 0 11.564416785288268
hcp maze32 169 2830 1 3.105644745884876
